# Задание для отбора на стажировку Департамент вычислительной биологии BIOCAD

"Мы часто обеспокоены тем, как изменения в белках влияют на их поведение. Проще всего судить об этом в терминах свободной энергии, поэтому в BIOCAD мы часто используем с молекулярную динамику. Поскольку белки большие и сложные молекулы, а собственный вычислительный кластер есть не у каждого, то в качестве тестового задания мы предлагаем менее «тяжеловесную» задачу. Предлагается оценить свободную энергию растворения в воде, для самой маленькой аминокислоты — глицина. Допустимо использовать любой движок молекулярной динамики. Если вы ранее не сталкивались с этим инструментом, можете использовать GROMACS (https://www.gromacs.org/).

В решении задания опишите, что было сделано для проведения расчёта, приведите результаты, которые получились. Попробуйте интерпретировать их и оценить, насколько они соответствуют реальным значениям."

Расчет свободной энергии удобен тем, что можно соотнести результаты вычисления с экспериментом. Если я правильно понимаю, так как при растворении не меняется кол-во вещества, $\Delta\nu$ = 0, энергия растворения равна энтальпии, $\Delta G$ = $\Delta H$. Сперва открыл [данные для глицина](https://webbook.nist.gov/cgi/cbook.cgi?Source=1959TAK%2FCHI84-88&Units=SI&Mask=1E9F) в базе National Institute of Standards and Technology (NIST), но энтальпии растворения здесь не было. В итоге нашел её в [статье](https://doi.org/10.1016/0040-6031(91)80390-5): $\Delta H^o _{sol} = 14.2 кДж/моль^{-1}$

Молекулярная динамика и Метод Монте Карло в стандартном виде не применяются, потому что для расчета свободной энергии используется соотношение между временем когда с-ма находилась в одном и другом состоянии $\Delta G = -RTlnK$, где $К = t1/t2$.
А МД и МК "не любят" те места фазового пр-ва, где энергия не минимальна, под "не любят" имеется в виду, что они находятся в этих состояниях с очень маленькой вероятностью и чтобы зафиксировать их, придется проводить динамику очень долго, это дорого

Технические характеристики и версии программ
* ОS: MacOS Sonoma build 23A344
* Chip: Apple M1
* Python: 3.11.6
* GROMACS: 2023.3-Homebrew
* Open Babel: 3.1.0
* ChemSketch: ACD/ChemSketch (Freeware) 2023.1.1

Импортирую все необходимые в дальнейшем модули


In [ ]:
!pip install pubchempy

  Preparing metadata (setup.py) ... done
  Created wheel for pubchempy: filename=PubChemPy-1.0.4-py3-none-any.whl size=13819 sha256=4a8bba207aeb23a7941ccbb6f35f2a960e7264415e5c703125c7d865922cb248
  Stored in directory: /root/.cache/pip/wheels/90/7c/45/18a0671e3c3316966ef7ed9ad2b3f3300a7e41d3421a44e799
Successfully built pubchempy


In [ ]:
import os
import shutil
from pathlib import Path
import pubchempy as pcp

Настраиваю рабочую директорию, создаю подпапки для входных и выходных файлов

In [ ]:
dir = Path('/Users/denischekalin/Desktop/biocad/simulation') # write your directory for simulation
os.chdir(dir)

Path('output_files').mkdir(exist_ok=True)
Path('input_files').mkdir(exist_ok=True)

input_files=dir.joinpath('input_files')
output_files=dir.joinpath('output_files')

Оптимальный вариант, это скачать уже готовую оптимизированную и протонированную структуру глицина c базы данных PubChem (CID 750)

In [ ]:
pcp.download('SDF', input_files/'gly.sdf', 'glycine', 'name', record_type='3d', overwrite=True)

Перевожу из sdf в pdb формат, это нужно, чтобы молекулу можно было подать pdb2gmx, команде gromacs, которая делает файл топологии на основе силового поля. Для этого использую Open Babel

In [8]:
!obabel 'input_files/gly.sdf' -O 'input_files/gly.pdb'

/bin/bash: line 1: obabel: command not found


Для проверки открыл файл gly.pdb в текстовом редакторе и в PyMOL, всё в порядке

![](https://drive.google.com/uc?export=view&id=1_CR7XVR-JLmClySWvZIagNGSk4n_FtE0)



![](https://drive.google.com/uc?export=view&id=1el5hWdLPhC85wT46GRIYT00t_q0Q9C3p)

Можно еще сделать и оптимизировать трехмерную структуру с помощью RDkit,

In [ ]:
!pip install rdkit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.5/30.5 MB 45.8 MB/s eta 0:00:00


In [ ]:
from rdkit import Chem
from rdkit.Chem import AllChem

glycine = Chem.MolFromSmiles("NCC(=O)O")
glycine = Chem.AddHs(glycine)
AllChem.EmbedMolecule(glycine)
AllChem.MMFFOptimizeMolecule(glycine)
Chem.MolToPDBFile(glycine, "input_files/gly_RD.pdb")

Со структурой всё в порядке, но если пользоваться ею дальше, то в pdb файле необходимо поменять названия атомов и вместо UNL подписать, что это GLY

![](https://drive.google.com/uc?export=view&id=1xDASxuc-fAywmrsrMpnTB8myuI668RsF)

![](https://drive.google.com/uc?export=view&id=1LQc3ebowe-YazMS1c783rBGm_FXjoOt8)

Или нарисовать с помощью ChemSketch, в этой программе хорошая оптимизация трехмерной структуры. Потом тоже перевести в pdb

In [ ]:
!obabel 'input_files/gly_CS.mol' -O 'input_files/gly_CS.pdb'

Тут всё в порядке

![](https://drive.google.com/uc?export=view&id=1jiVlvlzdt6n45g-ulwR74OnWrUlJPpME)

![](https://drive.google.com/uc?export=view&id=10f7ljGHLtMXWHIr8JbAHKAhuOSrS3Km9)

Как написал ранее, всё же если соединение есть в PubChem, оптимально взять его структуру оттуда.
Она уже протонирована и оптимизирована, pdb файл проверен, можно подавать его pdb2gmx.
Во многих силовых полях не было описания глицина как отдельного (начального и конечного) остатка. Работали только два поля: charmm27 и оplsaa. Charmm27, на мой взгляд, делает описание лучше, потому что на выходе в файле топологии (topol.top) суммарный заряд молекулы (qtot) получается равным нулю, тогда как в оplsaa он равен -0.11. Неудобно будет проводить динамику с таким условием, если оставить суммарный заряд моделируемой системы не нулевым, то ..., чтобы уравновесить заряды, обычно в раствор добавляют ионы (genion в громаксе), а как уравновесить не целый заряд? В итоге силовое поле (-ff) charmm27 и tip3p модель воды (громакс пишет, что для поля рекомендуется именно эта модель, я соглашусь)

In [ ]:
!gmx pdb2gmx -f './input_files/gly.pdb' -o './output_files/gly.gro'  -ff charmm27 -water tip3p #-ff oplsaa -water tip4p

Создаю коробку (додекаэдр) такого размера, чтобы минимальное расстояние между глицином в центре коробки и её краем  составляло 1,2 нм (такое значение брали для молекулы этанола в туториале на официальном сайте громакса, т.к. молекула глицина примерно того же размера, возьму такое же расстояние, и далее буду ориентироваться на значения из [туториала](https://tutorials.gromacs.org/free-energy-of-solvation.html)).

In [ ]:
!gmx editconf -f './output_files/gly.gro' -o './output_files/box.gro' -c -bt dodecahedron -d 1.2

Наливаем воду в коробку. Получается 490 молекул растворителя

In [ ]:
!gmx solvate -cp './output_files/box.gro' -cs -o './output_files/solvated.gro' -p 'topol.top'

Первоначальное размещение молекул воды не идеально, это связано с алгоритмом gmx solvate, сначала молекулы воды добавляются по всему объему коробки, а потом просто удаляются те, которые перекрылись с растворенным веществом. Поэтому  система может содержать участки пустого пространства и атомы, расположенные слишком близко друг к другу, что вызовет очень высокие межатомные силы. Поэтому предварительно нужно минимизировать энергию системы. А для этого подготовить .mdp файл с параметрами минимизации

In [ ]:
def write_mdp(mdp_string, mdp_name, directory):
    mdp_filename=os.path.join(directory,mdp_name)
    mdp_filehandle=open(mdp_filename,'w')
    mdp_filehandle.write(mdp_string)
    mdp_filehandle.close()

em_mdp=""";minimal mdp options for energy minimization
integrator               = steep
nsteps                   = 500
coulombtype              = pme
"""
write_mdp(em_mdp, 'em.mdp', output_files)

Затем em.mdp файл нужно предобработать с помощью grompp в файл выполнения em.tpr - файл, содержащий все данные, необходимые для выполнения минимизации, включая координаты атомов, топологию и протокол моделирования.

In [ ]:
!gmx grompp -f 'output_files/em.mdp' -c 'output_files/solvated.gro' -o 'output_files/em.tpr'

Запускаю моделирование минимизации энергии

In [ ]:
!gmx mdrun -v -deffnm 'output_files/em'

Для вычисления свободной энергии необходимо, чтобы система поддерживала постоянную температуру и давление
Поэтому необходимо уравновесить энергию в системе — достичь равномерного распределения энергии по системе. Для этого будут использоваться термостат и баростат (алгоритмы для сохранения соответствующих параметров). При релаксации с-мы образуется излишек кинетической энергии, повышается температура, самый простой способ контролировать температуру термостат с масштабированием скоростей. Баростат с C-масштабированием.
Так же как и при минимизации необходимо написать конфигурационный файл .mdp, с использование которого создается .tpr и передается в GROMACS для запуска моделирования.

In [ ]:
equil_mdp=""";equilibration mdp options
integrator               = md
nsteps                   = 100000
dt                       = 0.002
nstenergy                = 100
rlist                    = 1.1
nstlist                  = 10
rvdw                     = 1.1
coulombtype              = pme
rcoulomb                 = 1.1
fourierspacing           = 0.13
constraints              = h-bonds
tcoupl                   = v-rescale
tc-grps                  = system
tau-t                    = 0.5
ref-t                    = 300
pcoupl                   = C-rescale
ref-p                    = 1
compressibility          = 4.5e-5
tau-p                    = 1
gen-vel                  = yes
gen-temp                 = 300
"""
write_mdp(equil_mdp, 'equil.mdp', output_files)

In [ ]:
!gmx grompp -f 'output_files/equil.mdp' -c 'output_files/em.gro' -o 'output_files/equil.tpr'


In [ ]:
!gmx mdrun -deffnm 'output_files/equil'

Подготовка системы завершена

Так как не можем напрямую посчитать разницу в свободной энергии между глицином в вакууме и в воде, вводим в силовое поле параметр лямбда перед слагаемым отвечающим за взаимодействие между раствором и растворённым веществом, т.е. между водой и глицином. Будем этот параметр постепенно менять от 0 до 1, определять различные точки, каждая из которых будет указывать на "алхимическое" переходное состояние, между которыми уже сможем посчитать разницу в свободной энергии
Допустим растворение происходит при комнатной температуре 298K

In [ ]:
run_mdp="""; we'll use the sd integrator (an accurate and efficient leap-frog stochastic dynamics integrator) with 100000 time steps (200ps)
integrator               = sd
nsteps                   = 100000
dt                       = 0.002
nstenergy                = 1000
nstcalcenergy            = 50 ; should be a divisor of nstdhdl
nstlog                   = 5000
; cut-offs at 1.0nm
rlist                    = 1.1
rvdw                     = 1.1
; Coulomb interactions
coulombtype              = pme
rcoulomb                 = 1.1
fourierspacing           = 0.13
; Constraints
constraints              = h-bonds
; set temperature to 298K
tc-grps                  = system
tau-t                    = 2.0
ref-t                    = 298
; set pressure to 1 bar with a thermostat that gives a correct
; thermodynamic ensemble
pcoupl                   = C-rescale
ref-p                    = 1.0
compressibility          = 4.5e-5
tau-p                    = 5.0

; and set the free energy parameters
free-energy              = yes
couple-moltype           = Protein_chain_A
nstdhdl                  = 50 ; frequency for writing energy difference in dhdl.xvg, 0 means no ouput, should be a multiple of nstcalcenergy.
; these 'soft-core' parameters make sure we never get overlapping
; charges as lambda goes to 0
; soft-core function
sc-power                 = 1
sc-sigma                 = 0.3
sc-alpha                 = 1.0
; we still want the molecule to interact with itself at lambda=0
couple-intramol          = no
couple-lambda1           = vdwq
couple-lambda0           = none
init-lambda-state        = {}
; These are the lambda states at which we simulate
; for separate LJ and Coulomb decoupling, use
fep-lambdas              = 0.0 0.2 0.4 0.6 0.8 0.9 1.0
"""

Теперь нужно выполнить grompp и mdrun последовательно для каждой точки

In [ ]:
os.chdir(dir)
%cp topol.top output_files

In [ ]:
number_of_lambdas=7
for lambda_number in range(number_of_lambdas):
    lambda_directory=os.path.join(output_files,'lambda_{:0>2}'.format(lambda_number))
    Path(lambda_directory).mkdir(exist_ok=True)
    gro_file=os.path.join(output_files,'equil.gro')
    top_file=os.path.join(output_files,'topol.top')
    shutil.copy(gro_file,os.path.join(lambda_directory,'conf.gro'))
    shutil.copy(top_file,lambda_directory)
    write_mdp(run_mdp.format(lambda_number),'grompp.mdp',lambda_directory)
    %cd $lambda_directory
    !gmx grompp
    !gmx mdrun

In [ ]:
%cd ..

С помощью gmx bar посчитать разницу в свободной энергии между состояниями используя метод Bennett acceptance ratio (BAR). Сумма разниц энергий будет равна свободной энергии растворения.

In [ ]:
!gmx bar -b 100 -f lambda_0?/dhdl.xvg

Вывод: выходное значение свободной энергии варьируется от запуска к запуску, это неудивительно, т.к. система содержит стохастические составляющие. Но величина получилась на порядок больше значения, которое я ожидал: -244.43 кДж/моль